In [3]:
import pandas as pd
import numpy as np

# FULL UPDATED DATA: 26 real Nepali cricketers (camp squad + captains, Dec 2025)
data = [
    # Original + fixes
    ["ParasKhadka", 1,1,1,1,1, 1,1,1,1, 0,1, 0,0],  # Allr fast, ex-captain
    ["DipendraSinghAiree", 1,1,1,1,1, 1,1,1,0, 1,0, 0,1],  # Allr off-spin, young
    ["SandeepLamichhane", 1,1,1,1,1, 0,1,0,1, 1,0, 0,1],  # Leg-spin, ex-captain, young
    ["RohitPaudel", 1,1,1,1,1, 1,0,1,1, 1,0, 0,1],  # Bat allr off-spin, current captain, young
    ["SompalKami", 1,1,1,1,1, 0,1,1,0, 0,1, 0,0],  # Pace allr
    ["GyanendraMalla", 1,1,1,1,1, 1,0,0,1, 0,0, 0,0],  # Pure bat, ex-captain
    ["KaranKC", 1,1,1,1,1, 0,1,1,0, 0,1, 0,0],  # Pace allr
    ["AasifSheikh", 1,1,1,1,1, 1,0,0,0, 0,0, 1,1],  # WK bat, young
    ["KushalBhurtel", 1,1,1,1,1, 1,1,1,0, 1,0, 0,1],  # Bat leg-spin allr, young
    ["KushalMalla", 1,1,1,1,1, 1,1,1,0, 1,0, 0,1],  # Bat left-spin allr, young
    ["GulshanJha", 1,1,1,1,1, 1,1,1,0, 0,1, 0,1],  # Pace allr, young
    ["AarifSheikh", 1,1,1,1,1, 1,1,1,0, 0,1, 0,1],  # Pace allr, young
    ["LalitRajbanshi", 1,1,1,1,1, 0,1,0,0, 1,0, 0,0],  # Left orthodox spin
    ["AbinashBohara", 1,1,1,1,1, 0,1,0,0, 0,1, 0,0],  # Pace
    ["BinodBhandari", 1,1,1,1,1, 1,0,0,0, 0,0, 1,0],  # WK bat
    ["SundeepJora", 1,1,1,1,1, 1,0,0,0, 0,0, 0,1],  # Pure bat, young
    
    # NEW from T20 WC 2026 camp squad
    ["NandanYadav", 1,1,1,1,1, 0,1,1,0, 0,1, 0,1],  # Medium-fast allr, young
    ["LokeshBam", 1,1,1,1,1, 1,0,0,0, 0,0, 1,1],  # WK bat, young
    ["AadilAnsari", 1,1,1,1,1, 1,1,1,0, 0,1, 0,1],  # Bat fast allr, young
    ["SahabAlam", 1,1,1,1,1, 0,1,0,0, 1,0, 0,0],  # Left orthodox spin
    ["SherMalla", 1,1,1,1,1, 1,1,1,0, 1,0, 0,1],  # Off-spin allr, young
    ["BhimSharki", 1,1,1,1,1, 1,1,1,0, 1,0, 0,1],  # Bat off-spin allr, young
    ["IshanPandey", 1,1,1,1,1, 1,1,1,0, 1,0, 0,0],  # Left-spin allr
    ["BasirAhamad", 1,1,1,1,1, 1,1,1,0, 1,0, 0,1],  # Left orthodox allr, young
    ["PratishGC", 1,1,1,1,1, 0,1,0,0, 0,1, 0,1],  # Left medium pace, young
    ["RupeshKSingh", 1,1,1,1,1, 1,1,1,0, 0,1, 0,1]   # Medium-fast allr, young
]

columns = ["character","real","male","alive","cricketer","national_team",
           "batsman","bowler","all_rounder","captain_experience",
           "spin_bowler","fast_bowler","wicket_keeper","young_player"]

df = pd.DataFrame(data, columns=columns)
df.to_csv("nepali_cricket_akinator_full.csv", index=False)  # Save for later use

# Question dictionary (human-readable)
questions = {
    "batsman": "Is he primarily a batsman (top/middle order)?",
    "bowler": "Is he primarily a bowler?",
    "all_rounder": "Is he an all-rounder (bats AND bowls regularly)?",
    "captain_experience": "Has he captained Nepal's senior national team?",
    "spin_bowler": "Does he bowl spin?",
    "fast_bowler": "Does he bowl pace/fast-medium?",
    "wicket_keeper": "Is he a wicket-keeper?",
    "young_player": "Is he young/emerging (born 2000 or later)?"
}

features = list(questions.keys())  # Questions to ask

def entropy(series):
    """Compute entropy for player distribution."""
    if len(series) <= 1:
        return 0
    counts = series.value_counts()
    probs = counts / len(series)
    return -np.sum(probs * np.log2(probs))

def info_gain(df_remain, feature):
    """Compute information gain for a feature."""
    h_parent = entropy(df_remain['character'])
    yes_df = df_remain[df_remain[feature] == 1]
    no_df = df_remain[df_remain[feature] == 0]
    total = len(df_remain)
    if len(yes_df) == 0 or len(no_df) == 0:
        return 0
    h_yes = entropy(yes_df['character'])
    h_no = entropy(no_df['character'])
    return h_parent - (len(yes_df)/total * h_yes + len(no_df)/total * h_no)

# GAME LOOP
print("🤖 Nepali Cricket Akinator! Think of a player from the list.")
print("Players:", ", ".join(df['character'].tolist()))
print("Answer Y/N for questions. Let's guess! 🎯\n")

remain = df.copy()
steps = 0
while len(remain['character'].unique()) > 1:
    steps += 1
    # Find best feature
    best_feat = max(features, key=lambda f: info_gain(remain, f))
    q = questions[best_feat]
    print(f"Q{steps}: {q}")
    ans = input("Your answer (Y/N): ").strip().upper()
    if ans == 'Y':
        remain = remain[remain[best_feat] == 1]
    elif ans == 'N':
        remain = remain[remain[best_feat] == 0]
    else:
        print("Invalid! Enter Y/N.")
        continue
    
    possibles = remain['character'].tolist()
    print(f"Possibles left ({len(possibles)}): {', '.join(possibles)}\n")

guessed = remain['character'].iloc[0]
print(f"🎉 I guess: **{guessed}**! Correct? (Y/N)")
verify = input().strip().upper()
if verify == 'Y':
    print(f"✅ Guessed in {steps} questions! Play again?")
else:
    print("😅 Close! Add more data next time.")
print(df)  # Show full dataset

🤖 Nepali Cricket Akinator! Think of a player from the list.
Players: ParasKhadka, DipendraSinghAiree, SandeepLamichhane, RohitPaudel, SompalKami, GyanendraMalla, KaranKC, AasifSheikh, KushalBhurtel, KushalMalla, GulshanJha, AarifSheikh, LalitRajbanshi, AbinashBohara, BinodBhandari, SundeepJora, NandanYadav, LokeshBam, AadilAnsari, SahabAlam, SherMalla, BhimSharki, IshanPandey, BasirAhamad, PratishGC, RupeshKSingh
Answer Y/N for questions. Let's guess! 🎯

Q1: Does he bowl spin?


Your answer (Y/N):  yes


Invalid! Enter Y/N.
Q2: Does he bowl spin?


Your answer (Y/N):  y


Possibles left (11): DipendraSinghAiree, SandeepLamichhane, RohitPaudel, KushalBhurtel, KushalMalla, LalitRajbanshi, SahabAlam, SherMalla, BhimSharki, IshanPandey, BasirAhamad

Q3: Is he primarily a batsman (top/middle order)?


...

Possibles left (3): SandeepLamichhane, LalitRajbanshi, SahabAlam

Q4: Has he captained Nepal's senior national team?


Your answer (Y/N):  y


Possibles left (1): SandeepLamichhane

🎉 I guess: **SandeepLamichhane**! Correct? (Y/N)


 y


✅ Guessed in 4 questions! Play again?
             character  real  male  alive  cricketer  national_team  batsman  \
0          ParasKhadka     1     1      1          1              1        1   
1   DipendraSinghAiree     1     1      1          1              1        1   
2    SandeepLamichhane     1     1      1          1              1        0   
3          RohitPaudel     1     1      1          1              1        1   
4           SompalKami     1     1      1          1              1        0   
5       GyanendraMalla     1     1      1          1              1        1   
6              KaranKC     1     1      1          1              1        0   
7          AasifSheikh     1     1      1          1              1        1   
8        KushalBhurtel     1     1      1          1              1        1   
9          KushalMalla     1     1      1          1              1        1   
10          GulshanJha     1     1      1          1              1        1   
11